In order to properly work with our data we need to know where each image is. We do this by plate solving; the easiest way for us to do this is to uplaod each of our images to nova.astronomy.net. Doing this by hand is a bit tedious, so this notebook helps do it automatically.

The general process is fairly simple:
1. Use our API key to let the server know we're allowed to send it a bunch of things and that it can trust us.
2. Make a list of every file that we've placed in a specific 'input' folder.
3. Upload each of those files to Astrometry, and save the special number it assigns that submission.
4. Wait a bit to make sure it has time to process. (If uploading lots of files, this should not be a problem in the slightest, as the time it takes us to uplaod is enough for it to process)
5. Download a new, enchanced, fits file from Astrometry into the folder this notebook is in.
6. Move all those downloaded files to their final home; the 'output' folder.

# Import Statements and Required Functions

In [ ]:
import sys
#My folder is at: /Users/aidanmcclung/Desktop/Summer_Exoplanets
sys.path.append('/Users/aidanmcclung/Desktop/Summer_Exoplanets') #this lets python know to look here for import statements

In [ ]:

#   Set whichever one of these is applicable to you. Hopefully it's the second one. ;p
#import client #This is a .py file you need to have in the folder with this notebook
import QAOP.client as client #This is a .py file you need to have in the QAOP folder

import os #OS is essentially just the basic terminal comand line interface tools in python. 
#We need it to make folders and move our files around.

#We need two packages in order to do our web operations:
import requests #The default python webserver interfacing package
import json #The python JavaScript Object Notation interfacing package

from IPython.display import clear_output #to help with the progress bars :)

#make sure there are input and output folders to use
for rq_dir in ["input","output"]:
    if not os.listdir().count(rq_dir): print(rq_dir,": dir does not exist, creating one"); os.mkdir(rq_dir)

## Function Definitions

I created each of these functions, and have attempted to add enough comments to describe what they do.
Knowing those specifics isn't necessary though, and if you're not interested you can absolutely **run each of these cells once** and move on to the **Main Program**

In [ ]:
def findFileName(fn,debug=False):
    if fn[-4:] == ".fit": 
        name = fn[:-4]
        if debug: print(".fit file detected.  Saving filename as:",name)
    if fn[-5:] == ".fits": 
        name = fn[:-5]
        if debug: print(".fits file detected. Saving filename as:",name)
    return name

def getSubID(fn,debug=False):
    #Now we need to upload the file to astrometry.net
    # we do this using a method in the client class: upload()
    #This then returns a json status, but we only care about the info it gives us about where it went,
    # ie the "Submission ID", which we take out of the result.
    SUBIDforThis = cAPI.upload('input/'+fn)["subid"]
    if debug: print(f"SUBID for {fn} is {SUBIDforThis}")
    return SUBIDforThis

def submitFile(fn,debug=False):
    name = findFileName(fn) #detect whether .fits or .fit so we can get the right characters for the name
    if debug: print("Submitting File:",name)
    subid = getSubID(fn) #This function is where we upload the file
    nameSubs[name] = subid #We add what it was saved as to our own dictionary, so we know whats what.
    return name #We return the name and add it to a list with this call, 
#which we then use later to iterate through all the things we uploaded

In [ ]:
def getJobID(SubID,debug=False):
    #When Astrometry does it's thing, we submit our image, but it hasn't been solved.
    #Once it has been solved, it's assigned a "jobID" which we need in order to access the results.
    R = requests.post('http://nova.astrometry.net/api/submissions/'+str(SubID))
    #R will be an object with the response to the web post we sent.
    #print(R.json()) #Uncomment to see this response
    JOBIDSforThis = R.json()["jobs"] #A submission *may* have multiple jobs. 
    JOBIDforThis = JOBIDSforThis[0] #We only care about the first one that was done
    if debug: print(JOBIDforThis)
    return JOBIDforThis


from urllib.request import urlretrieve as saveFromWeb
def saveNewFits(JOBIDforThis,name):
    saveFromWeb("https://nova.astrometry.net/new_fits_file/"+str(JOBIDforThis), name+".fits")
    return name+".fits"

def retrieveFile(name,debug=False):
    #print("--------:",nameSubs)
    subid = nameSubs[name] #First retrieve the SubID for this file
    jobid = getJobID(subid) #Next we acquire the JobID for it's succesful result
    newName = saveNewFits(jobid,name) #We use the JobID to get the file from astrometry, 
    #and then we need the name in order to save it as the right name
    return newName

# Main Program

In [ ]:
#We need to now log in to the API, this should be... possible...

#USERS NEED TO ENTER AN API KEY!!!!

APIKEY = input("Please enter your API Key")
#webboctjikkepcfj #Aidan's API Key

cAPI = client.Client()
cAPI.login(APIKEY)

#webboctjikkepcfj #Aidan's API Key

In [ ]:
#once we're logged in, we need to get all the files in the input directory.
inputFiles_all = os.listdir("input")
#The program will pull in ANY files that are there; 

#sometimes there are hidden os files or such we don't want, so we need to filter those out.
#print(inputFiles_all)
inputFiles = []
for file in inputFiles_all:
    if file[-4:] == '.fit': inputFiles.append(file)
#print(inputFiles)

numFiles = len(inputFiles)
print(numFiles,'Files:\n',inputFiles)

In [ ]:
#This dictionary will contain the mapping between the names and what submission IDs they were assigned 
# when they were uploaded.
nameSubs = {}
#We define it in it's own cell so that it doesn't ever get deleted/cleared accidentally 
# if we need to run a part of the program again.
names = []

In [ ]:
#Set these two here to maintain progress upon hitting an error. NOT FULLY FUNCTIONAL, USE WITH DISCRETION
upload_progress = 0 
checkFiles = []

In [ ]:
while upload_progress < numFiles:
    file = inputFiles[upload_progress]
    #submit file
    name = submitFile(file)
    #save name
    names.append(name)
    checkFiles.append(file)
    #print(file)
    #Take care of some of the feedback stuff so we know how it's doing
    upload_progress += 1
    clear_output(wait=False); print('Upload Status:',upload_progress,'of',numFiles, "   Most Recent File (Unordered):",name)
    

#And lastly we just run a check to make sure that all of them get uploaded right. you should see nothing
for file in inputFiles:
    if not checkFiles.count(file): print(file,"Was not present in the redundancy list.")

In [ ]:
savedFiles = []
download_progress = 0
numDown = len(names)

In [ ]:
while download_progress < numDown:
    name = names[download_progress]
    if not savedFiles.count(name): #make sure we don't download twice if we added a name twice accidentally
        downFile = retrieveFile(name)
        savedFiles.append(downFile)
    
    #Take care of some of the feedback stuff so we know how it's doing
    download_progress += 1
    clear_output(wait=False); print('Download Status:',download_progress,'of',numDown, "  Most Recent File (Unordered):",savedFiles[-1])

In [ ]:
#uncomment either of these if you need to check something. 
#You can copy the 7-digit number and paste it into the following link to look at the image in a web browser.
#  https://nova.astrometry.net/user_images/[subid]#annotated

#print(nameSubs)
#print(len(nameSubs))

In [ ]:
#if you want to look at what it saved:
#print(savedFiles)

In [ ]:
#If we ran the loop multiple times, when we redownloaded a file, it would have overwrote the old one.
#However, it would still get readded to the list. So, we want to take out any duplicates, 
#which we can do by a sneaky trick; casting a dict and then back to a list:
savedFiles = list(dict.fromkeys(savedFiles)) #remove duplicates
#Note: this bug can be fixed by putting the savedFiles list in the same cell as it's loop, but then we can't continue if interrupted

In [ ]:
#We can only download files to the same location as this python file, for cybersecurity reasons
# After we've downloaded all the files into our main folder, we want to put them into their own 
# folder, which is what this lil loop here does

#print(savedFiles) #troubleshooting
for fn in savedFiles:
    #if fn == '060.fits': continue #There was an error and 060.fits was corrupted. Replicate if you encounter problems
    os.rename(fn,'output/'+fn)

# Verification (Optional)

To check that everything worked, you can compare the following four lists, and their lengths.

If you're interested in seeing what changed by doing this process, you can do so with the cells that follow.

In [ ]:
#We may want to check that we did all the right things, by comparing the following:

#print(nameSubs.keys()) #everything we uploaded
#print(names) #Everything we uploaded in the most recent submission
#print(savedFiles) #Everything we downloaded in the most recent retrieval
#print(os.listdir('output')) #Everything we have now that we're done

In [ ]:
#These two packages are only used to check on whether the program worked or not.
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
print("Before Astrometry:")
print(" ")

inputFileName = "k009.fit" #Change this to be what file you'd like to check
with fits.open("input/" + inputFileName) as f:
    print(f[0].header)

print(" ")
print("------------------------------")
print("After Astrometry:")
print(" ")

outputFileName = inputFileName + "s" #change this if the only change wasn't .fit -> .fits
with fits.open("output/"+outputFileName) as f:
    print(f[0].header)

In [ ]:
print("Before Astrometry:")
print(" ")

inputFileName = "k009.fit" #Change this to be what file you'd like to check
with fits.open("input/" + inputFileName) as f:
    w = WCS(f[0].header)
    print(w)

print(" ")
print("------------------------------")
print("After Astrometry:")
print(" ")

outputFileName = inputFileName + "s" #change this if the only change wasn't .fit -> .fits
with fits.open("output/"+outputFileName) as f:
    w = WCS(f[0].header)
    print(w)